# Answer

In [51]:
import math

class SentiAnalyzer_eunseok:

    # Make the method signature to accept "sentidata" and "word"
    def __init__(self):
        print('This is a senti analyzer')

    def probWordPositiveAndNegative(self, sentidata, word, idxWord):
        pointedWord = word[idxWord]
        reviews = [int(float(row[-1])) for row in sentidata]
        occurrence = [int(float(row[idxWord])) for row in sentidata]

        # Calculate the number of positive review occurrence with the pointed word, and assign the calculated value to 'positive'
        positive = 0
        for i in range(len(occurrence)):
            if reviews[i] == 1:
                positive = positive + occurrence[i]

        # Calculate the number of positive reviews from the entire review set
        numPositiveReviews = sum(reviews)

        # Calculate the number of negative review occurrence with the pointed word, and assign the calculated value to 'negative'
        negative = 0
        for i in range(len(occurrence)):
            if reviews[i] == 0:
                negative = negative + occurrence[i]

        rowCount = len(sentidata)
        positiveProb = float(positive) / float(numPositiveReviews)
        negativeProb = float(negative) / float(rowCount - numPositiveReviews)

        if positiveProb == 0:
            positiveProb = 0.00001
        if negativeProb == 0:
            negativeProb = 0.00001
        return pointedWord, positiveProb, negativeProb

    def probPositiveAndNegative(self, sentidata):
        positive = sum([int(float(row[-1])) for row in sentidata])
        numReviews = len(sentidata)
        negative = numReviews - positive
        positiveProb = float(positive) / float(numReviews)
        negativeProb = float(negative) / float(numReviews)
        return positiveProb, negativeProb

    def findUsedWords(self, sentidata, word, idx):
        # Return the index of the used words in 'idx'th review
        temp = [int(float(x)) for x in sentidata[idx][:-1]]
        idxUsedWords = [index for index, value in enumerate(temp) if value == 1]
        # Return the actual words in 'idx'th review
        usedWords = [word[idx] for idx in idxUsedWords]
        return idxUsedWords, usedWords

    def runAnalysis(self, sentidata, word, idxReview):
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(sentidata, word, idxReview)

        # Make a for-loop to run from the first word to the last word
        for i in range(len(idxUsedWords)):
            # get the first word from the used word set
            idxWord = idxUsedWords[i]
            # calculate the word's probability to be positive or negative
            pointedWord, positive, negative = self.probWordPositiveAndNegative(sentidata, word, idxWord)
            probLogPositive += math.log(positive)
            probLogNegative += math.log(negative)

        positiveProb1, negativeProb1 = self.probPositiveAndNegative(sentidata)
        probLogPositive += math.log(positiveProb1)
        probLogNegative += math.log(negativeProb1)

        if probLogPositive > probLogNegative:
            sentiment = 'Positive'
            print('Positive')
        else:
            sentiment = 'Negative'
            print('Negative')

        return probLogPositive, probLogNegative, sentiment

In [55]:
# from SentiAnalyzer import *
import pandas as pd

word_df = pd.read_csv('word.csv', header=None)
sentidata_df = pd.read_csv('sentidata.csv', header=None)

# DataFrame을 리스트로 변환
word = word_df.values.tolist()
sentidata = sentidata_df.values.tolist()

s = SentiAnalyzer_eunseok()
# 1)
print('1)')
s.runAnalysis(sentidata, word, 10)
# 2)
print('2)')
s.runAnalysis(sentidata, word, 120)
# 3)
print('3)')
pos, neg, sent = s.runAnalysis(sentidata, word, 120)
print(pos)
print(neg)
print(sent)

This is a senti analyzer
1)
Positive
2)
Negative
3)
Negative
-914.7005035681955
-456.4480969908263
Negative


# Let's start how to make Naive Bayes Classifier

## Load dataset

In [1]:
import pandas as pd

word_df = pd.read_csv('word.csv', header=None)
sentidata_df = pd.read_csv('sentidata.csv', header=None)

word_df.head()

,0
0,holes
1,must
2,top_secret
3,he
4,center


In [2]:
sentidata_df.head() # row: product, column: word

,0,1,2,3,4,5,6,7,8,9,...,28615,28616,28617,28618,28619,28620,28621,28622,28623,28624
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
sentidata_df.values

array([[1., 1., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## word(단어), sentidata(상품 리뷰 BoW) 리스트로 변환

In [4]:
for i in range(len(sentidata_df)):
    print(sentidata_df.iloc[i].value_counts()) # 2 이상이 없는 것으로 보아 단어의 출현 횟수가 아니라 단어의 출현 여부임

0.0    28359
1.0      266
Name: 0, dtype: int64
0.0    28452
1.0      173
Name: 1, dtype: int64
0.0    28553
1.0       72
Name: 2, dtype: int64
0.0    28510
1.0      115
Name: 3, dtype: int64
0.0    28268
1.0      357
Name: 4, dtype: int64
0.0    28284
1.0      341
Name: 5, dtype: int64
0.0    28383
1.0      242
Name: 6, dtype: int64
0.0    28518
1.0      107
Name: 7, dtype: int64
0.0    28404
1.0      221
Name: 8, dtype: int64
0.0    28419
1.0      206
Name: 9, dtype: int64
0.0    28202
1.0      423
Name: 10, dtype: int64
0.0    28494
1.0      131
Name: 11, dtype: int64
0.0    28285
1.0      340
Name: 12, dtype: int64
0.0    28571
1.0       54
Name: 13, dtype: int64
0.0    28525
1.0      100
Name: 14, dtype: int64
0.0    28437
1.0      188
Name: 15, dtype: int64
0.0    28511
1.0      114
Name: 16, dtype: int64
0.0    28494
1.0      131
Name: 17, dtype: int64
0.0    28368
1.0      257
Name: 18, dtype: int64
0.0    28498
1.0      127
Name: 19, dtype: int64
0.0    28554
1.0       71
Name

In [5]:
# DataFrame을 리스트로 변환
word = word_df.values.tolist()
sentidata = sentidata_df.values.tolist()

word # vocabulary list

[['holes'],
 ['must'],
 ['top_secret'],
 ['he'],
 ['center'],
 ['other_civilans'],
 ['the_pacific'],
 ['the_navy'],
 ['a_lot'],
 ['surface_must'],
 ['this_book'],
 ['man_named'],
 ['<num>_feet'],
 ['would_strongly'],
 ['put_down'],
 ['norman_johnson'],
 ['lawes'],
 ['a_top'],
 ['the_support'],
 ['ten'],
 ['on_random'],
 ['typhoon'],
 ['a_phycologist'],
 ['pressure'],
 ['actually_an'],
 ['a_day'],
 ['johnson_is'],
 ['strange'],
 ['civilans_to'],
 ['explored'],
 ['support'],
 ['pacific_ocean'],
 ['pressure_to'],
 ['back_some'],
 ['read'],
 ['however_on'],
 ['it_still'],
 ['stuck'],
 ['a_remote'],
 ['american'],
 ['find_out'],
 ['not_have'],
 ['other_crichton'],
 ['analysis.i'],
 ['black_holes'],
 ['after_a'],
 ['misssion'],
 ['some_strange'],
 ['half_mile'],
 ['<num>_navy'],
 ['actually'],
 ['remote_location'],
 ['the_story'],
 ['behavior'],
 ['civilans'],
 ['research_that'],
 ['michael_crichton'],
 ['excellant_novel'],
 ['strongly_recommend'],
 ['some'],
 ['michael'],
 ['does_not'],
 ['

In [6]:
sentidata # product 별 BoW -> 빈도가 아니라 출현 여부로 나타냄

[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,

## 특정 단어의 긍정/부정일 때 출현 확률 구하기

In [7]:
pointedWord = word[0] # holes
reviews = [int(row[-1]) for row in sentidata] # 긍/부정 리뷰 데이터

In [8]:
occurrence = [int(row[0]) for row in sentidata] # product별 holes 출현 여부
sum(occurrence)

3

In [9]:
# Calculate the number of positive review occurrence with the pointed word, and assign the calculated value to 'positive'
positive = 0
for i in range(len(occurrence)):
    if reviews[i] == 1: # 긍정일 때
        positive += occurrence[i] # holes 출현 개수

# Calculate the number of positive reviews from the entire review set
numPositiveReviews = reviews.count(1)

# Calculate the number of negative review occurrence with the pointed word, and assign the calculated value to 'negative'
negative = 0
for i in range(len(occurrence)):
    if reviews[i] == 0: # 부정일 때
        negative += occurrence[i] # holes 출현 개수, negative += -(occurrence[i]-1) -> 출현 안한 개수

In [10]:
print(positive) # 긍정일 때 holes 출현 개수
print(negative) # 부정일 때 holes 출현 개수

1
2


In [ ]:
rowCount = len(sentidata)
positiveProb_w = positive / numPositiveReviews # 긍정일 때 holes 단어 출현 확률
negativeProb_w = negative / (rowCount - numPositiveReviews) # 부정일 때 holes 단어 출현 확률

print(rowCount)
print(positiveProb_w)
print(negativeProb_w)

198
0.014285714285714285
0.015625


### Laplacian Smoothing 대체 
특정 단어가 하나도 출현하지 않았을 때 확률이 0이 되는 것을 막기 위해 Laplacian Smoothing을 사용  
-> 여기서는 Laplacian 대신 매우 작은 확률인 0.0001을 사용함.

In [12]:
''' if positiveProb_w == 0:
        positiveProb_w = 0.00001
    if negativeProb_w == 0:
        negativeProb_w = 0.00001
    return pointedWord, positiveProb_w, negativeProb_w '''

' if positiveProb_w == 0:\n        positiveProb_w = 0.00001\n    if negativeProb_w == 0:\n        negativeProb_w = 0.00001\n    return pointedWord, positiveProb_w, negativeProb_w '

### 전체 데이터셋의 긍정/부정 비율 구하기

In [13]:
def probPositiveAndNegative(sentidata):
    positive = sum([int(row[-1]) for row in sentidata])
    numReviews = len(sentidata)
    positiveProb = positive / numReviews
    negativeProb = 1 - positiveProb
    return positiveProb, negativeProb

In [14]:
probPositiveAndNegative(sentidata)

(0.35353535353535354, 0.6464646464646464)

In [30]:
def findUsedWords(sentidata, word, idx):
    # Return the index of the used words in 'idx'th review
    temp = [int(x) for x in sentidata[idx][:-1]] # 특정 product의 BoW
    idxUsedWords = [index for index, value in enumerate(temp) if value == 1] # 출현한 word의 index 가져오기
    # Return the actual words in 'idx'th review
    usedWords = [word[idx] for idx in idxUsedWords] # word
    return idxUsedWords, usedWords

In [58]:
findUsedWords(sentidata, word, 0)

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  15

In [57]:
def runAnalysis(sentidata, word, idxReview): # 10
    probLogPositive = 0
    probLogNegative = 0
    idxUsedWords, usedWords = findUsedWords(sentidata, word, idxReview) # 10번째 product 리뷰에 출현한 word index와 word

    # Make a for-loop to run from the first word to the last word
    for i in range(len(usedWords)):
        # get the first word from the used word set
        idxWord = idxUsedWords[i]
        # calculate the word's probability to be positive or negative
        pointedWord, positive, negative = probWordPositiveAndNegative(sentidata, word, idxWord)
        probLogPositive += math.log(positive)
        probLogNegative += math.log(negative)

    positiveProb1, negativeProb1 = self.probPositiveAndNegative(sentidata)
    probLogPositive += math.log(positiveProb1)
    probLogNegative += math.log(negativeProb1)


In [ ]:
if probLogPositive > probLogNegative:
    sentiment = 'Positive'
    print('Positive')
else:
    sentiment = 'Negative'
    print('Negative')
return probLogPositive, probLogNegative, sentiment